In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

# Kameradan Görüntü Alma

In [ ]:
# Kameraya erişim sağlamak için VideoCapture objesini oluştur
# 0, bilgisayarın varsayılan kamerasını ifade eder
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Kameraya erişilemiyor.")
    exit()

# Döngü içinde görüntüyü sürekli olarak yakala ve ekrana göster
#while True:
    # Kameradan bir kare al
ret, frame = cap.read()

# Eğer görüntü başarıyla alınmadıysa döngüyü kır
if not ret:
    print("Görüntü alınamıyor.")
    #break
    exit()

# Görüntüyü ekrana yazdır

cv2.imshow('Camera', frame)

print(frame.shape)
# 'q' tuşuna basıldığında döngüden çık
if cv2.waitKey(1) & 0xFF == ord('q'):
    #break
    exit()
# İşlem bitince kamera ve pencereleri serbest bırak
cap.release()
cv2.destroyAllWindows()


# Gaussian ve Sobel Uygulama

In [ ]:
# Converting BGR to RGB
#frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

frame_red = frame[:, :, 2]
frame_green = frame[:, :, 1]
frame_blue = frame[:, :, 0]
frame_rgb = np.dstack((frame_red, frame_green, frame_blue))

plt.imshow(frame_rgb)
plt.title("Frame")
plt.show()

In [ ]:
def rgb_to_grayscale(image):
    height, width = image.shape[0:2]
    grayscale_image = np.zeros((height, width), dtype=image.dtype)

    for y in range(height):
        for x in range(width):
            red, green, blue = image[y][x]
            gray = np.uint8(red * 0.299 + green * 0.5870 + blue * 0.1140)
            grayscale_image[y][x] = gray
            
    return grayscale_image

In [ ]:
frame_grayscale = rgb_to_grayscale(frame)

plt.imshow(frame_grayscale, cmap='gray')
plt.title("Frame_grayscale")
plt.show()

In [ ]:
def gaussian_filter(image, kernel_size = 3, sigma = 1):
    sigma_squared = sigma ** 2
    pi = math.pi
    e_value = math.e
    center = kernel_size // 2
    gaussian_kernel = np.zeros((kernel_size, kernel_size), dtype=np.float32)

    for y in range(-center, center + 1):
        for x in range(-center, center + 1):
            Gxy = (1 / (2 * pi * sigma_squared)) * (e_value ** ( -((x ** 2 + y ** 2) / (2 * sigma_squared))))
            gaussian_kernel[y + center][x + center] = Gxy

    
    gaussian_kernel /= gaussian_kernel.sum()

    height, width = image.shape
    blurred_image = np.zeros((height, width), dtype=np.uint8)
    
    padded_image = np.zeros((height + center * 2, width + center * 2), dtype=np.uint8)

    padded_image = np.pad(image, pad_width=center, mode='edge')
#    for row in range(0, height + 2, 1):
#        for column in range(0, width + 2, 1):
#            if column == 0 and row == 0:
#                padded_image[0][0] = image[0][0]
#            elif (column == width + 1) and (row == height + 1):
#                padded_image[row][column] = image[height - 1][width - 1]
#            elif (column == width + 1) and row == 0:
#                padded_image[0][column] = image[0][width - 1]
#            elif column == 0 and (row == height + 1):
#                padded_image[row][0] = image[height - 1][0]
#            elif row == 0:
#                padded_image[0][column] = image[0][column - 1]
#            elif column == 0:
#                padded_image[row][0] = image[row - 1][0]
#            elif row == height + 1:
#                padded_image[row][column] = image[row - 2][column - 1]
#            elif column == width + 1:
#                padded_image[row][column] = image[row - 1][column - 2]
#            else:
#                padded_image[row][column] = image[row - 1][column - 1]    

    for row in range(1, height + 1, 1):
        for column in range(1, width + 1, 1):
            total = 0
            for y in range(kernel_size):
                for x in range(kernel_size):
                    total += gaussian_kernel[y][x] * padded_image[row + y - 1][column + x - 1]

            blurred_image[row - 1][column - 1] = np.clip(round(total), 0, 255)

    print(blurred_image)
    return blurred_image

In [ ]:
filtered_grayscale_frame = gaussian_filter(frame_grayscale, kernel_size = 7, sigma = 1)

plt.imshow(filtered_grayscale_frame, cmap='gray')
plt.title("filtered_grayscale_frame")
plt.show()

In [ ]:
def sobel_filter(image):
    sobel_filter_x = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])
    sobel_filter_y = np.array([[-1, -2, -1],
                               [0, 0, 0],
                               [1, 2, 1]])
    kernel_size = 3
    
    height, width = image.shape
    filtered_image = np.zeros((height, width), dtype=np.uint8)
    
    padded_image = np.zeros((height + 2, width + 2), dtype=np.uint8)
    magnitudes = np.zeros((height, width), dtype=np.float32)

    padded_image = np.pad(image, pad_width=1, mode='edge')
#    for row in range(0, height + 2, 1):
#        for column in range(0, width + 2, 1):
#            if column == 0 and row == 0:
#                padded_image[0][0] = image[0][0]
#            elif (column == width + 1) and (row == height + 1):
#                padded_image[row][column] = image[height - 1][width - 1]
#            elif (column == width + 1) and row == 0:
#                padded_image[0][column] = image[0][width - 1]
#            elif column == 0 and (row == height + 1):
#               padded_image[row][0] = image[height - 1][0]
#            elif row == 0:
#                padded_image[0][column] = image[0][column - 1]
#            elif column == 0:
#                padded_image[row][0] = image[row - 1][0]
#            elif row == height + 1:
#                padded_image[row][column] = image[row - 2][column - 1]
#            elif column == width + 1:
#                padded_image[row][column] = image[row - 1][column - 2]
#            else:
#                padded_image[row][column] = image[row - 1][column - 1]    

    for row in range(1, height + 1, 1):
        for column in range(1, width + 1, 1):
            total_x = 0
            total_y = 0
            for y in range(kernel_size):
                for x in range(kernel_size):
                    total_x += sobel_filter_x[y][x] * padded_image[row + y - 1][column + x - 1]
                    total_y += sobel_filter_y[y][x] * padded_image[row + y - 1][column + x - 1]
            magnitude = np.sqrt(total_x**2 + total_y**2)
            magnitudes[row - 1][column - 1] = magnitude

    threshold = 40   
    min_magnitude = magnitudes.min()
    max_magnitude = magnitudes.max()

    if max_magnitude - min_magnitude > 0:
        normalized_magnitudes = (magnitudes - min_magnitude) / (max_magnitude - min_magnitude) * 255
        filtered_image = np.where(normalized_magnitudes > threshold, normalized_magnitudes, 0).astype(np.uint8)
    else:
        filtered_image = np.zeros_like(magnitudes, dtype=np.uint8)

    return filtered_image

In [ ]:
edge_dedection_grayscale_frame = sobel_filter(filtered_grayscale_frame)

plt.imshow(edge_dedection_grayscale_frame, cmap='gray')
plt.title("edge_dedection_grayscale_frame")
plt.show()

# Geometrik İlişkilere Dayalı Yüz Algılama

In [ ]:
def findContours(binary_image):
#    height, width = binary_image.shape
#    white_pixels = []

#    for y in range(height):
#        for x in range(width):
#            if binary_image[y, x] == 255:
#                white_pixels.append((x, y))

    height, width = binary_image.shape
    visited = np.zeros((height, width), dtype=bool)
    contours = []

    def DFS(x, y, current_contour):
        if x < 0 or y < 0 or x >= width or y >= height:
            return

        if visited[y, x] or binary_image[y, x] == 0:
            return

        visited[y, x] = True
        current_contour.append((x, y))

        for Nx, Ny in [(-1, 0), (-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1)]:
            DFS(x + Nx, y + Ny, current_contour)

    for y in range(height):
        for x in range(width):
            if binary_image[y, x] == 255 and not visited[y, x]:
                current_contour = []
                DFS(x, y, current_contour)

                if current_contour:
                    contours.append(current_contour)
    return contours
    
    compresed_contours = []
    for contour in contours:
        compresed_contour = []
        previous_direction = None
    
        for i in range(len(contour)):
            dx = contour[i][0] - contour[i - 1][0]
            dy = contour[i][1] - contour[i - 1][1]
            direction = (dx, dy)

            if direction != previous_direction:
                compresed_contour.append(contour[i-1])
                previous_direction = direction

        compresed_contour.append(contour[-1])
        compresed_contours.append(compresed_contour)

    def calculate_bounding_box(contour):
        x_min = min(point[0] for point in contour)
        x_max = max(point[0] for point in contour)
        y_min = min(point[1] for point in contour)
        y_max = max(point[1] for point in contour)
        return (x_min, y_min, x_max, y_max)

    def is_inside(inner_box, outer_box):
        inner_x_min, inner_y_min, inner_x_max, inner_y_max = inner_box
        outer_x_min, outer_y_min, outer_x_max, outer_y_max = outer_box
        return (inner_x_min > outer_x_min and inner_y_min > outer_y_min and inner_x_max < outer_x_max and inner_y_max < outer_y_max)

    def calculate_area(contour):
        n = len(contour)
        area = 0
        for i in range(n):
            x1, y1 = contour[i]
            x2, y2 = contour[(i + 1) % n]
            area += x1 * y2 - y1 * x2
        return abs(area) / 2

    def sort_contours(contours):
        areas = []
        for index, contour in enumerate(contours):
            area = calculate_area(contour)
            areas.append((index, area))

        sorted_areas = sorted(areas, key=lambda item: item[1], reverse=false)

        sorted_contours =  []
        for item in sorted_areas:
            contour_index = item[0]
            sorted_contours.append(contours[contour_index])

        return sorted_contours


    def calculate_hierarchy(contours):
        sorted_contours = sort_contours(contours)
        bounding_boxes = [calculate_bounding_box(contour) for contour in sorted_contours]
        hierarchy = [{"parent": -1, "children": []} for _ in sorted_contours]

        for i, inner_box in enumerate(bounding_boxes):
            for j, outer_box in enumerate(bounding_boxes):
                if i != j and is_inside(inner_box, outer_box):
                    if hierarchy[i]["parent"] == -1:
                        hierarchy[i]["parent"] = j
                        hierarchy[j]["children"].append(i)
    
        return sorted_contours, hierarchy

In [ ]:
def boundingRect(contour):
    x_coords = [point[0] for point in contour]
    y_coords = [point[1] for point in contour]

    x_min = min(x_coords)
    x_max = max(x_coords)
    y_min = min(y_coords)
    y_max = max(y_coords)

    width = x_max - x_min
    height = y_max - y_min

    print(x_min, x_max, y_min, y_max, width, height)

    return x_min, y_min, width, height

In [ ]:
def find_eyes(top_half):
    threshold = 50
    # Pixel değeri threshold'dan büyükse pixel'i 255, küçükse 0 yapar.
    binary_top = np.where(top_half > threshold, 255, 0).astype(np.uint8)

    # Gözlerin olabileceği yuvarlak bölgeleri bul
#    contours, _ = cv2.findContours(binary_top, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = findContours(binary_top)
    eyes = []
    for contour in contours:
        print("****"*10)
        print(contour)
        print("****"*10)
        x, y, w, h = boundingRect(contour)
        if 10 < w < 150 and 10 < h < 150:  # Göz boyutları için mantıklı bir aralık
            aspect_ratio = w / h 
            eyes.append((x, y, w, h))
    return eyes

In [ ]:
import numpy as np

# Bounding box hesaplama
def calculate_bounding_box(contour):
    x_min = min(point[0] for point in contour)
    x_max = max(point[0] for point in contour)
    y_min = min(point[1] for point in contour)
    y_max = max(point[1] for point in contour)
    return (x_min, y_min, x_max, y_max)

# İç içe kontrolü
def is_inside(inner, outer):
    inner_x_min, inner_y_min, inner_x_max, inner_y_max = inner
    outer_x_min, outer_y_min, outer_x_max, outer_y_max = outer
    return (inner_x_min > outer_x_min and inner_y_min > outer_y_min and
            inner_x_max < outer_x_max and inner_y_max < outer_y_max)


def calculate_area(contour):
    """
    Shoelace algoritmasını kullanarak verilen bir konturun alanını hesaplar.
    """
    n = len(contour)
    area = 0
    for i in range(n):
        x1, y1 = contour[i]
        x2, y2 = contour[(i + 1) % n]  # Bir sonraki nokta (son nokta için ilk noktaya döner)
        area += x1 * y2 - y1 * x2
    return abs(area) / 2  # Alanın mutlak değeri

def sort_contours_by_area(contours):
    """
    Konturları kapladıkları alana göre büyükten küçüğe sıralar.
    """
    areas = [(i, calculate_area(contour)) for i, contour in enumerate(contours)]
    # Alanlara göre sıralama (büyükten küçüğe)
    sorted_areas = sorted(areas, key=lambda x: x[1], reverse=False)
    # Sıralanmış konturların indekslerini döndür
    return [contours[i[0]] for i in sorted_areas]


# Hiyerarşi hesaplama
def calculate_hierarchy(contours):
    contours = sort_contours_by_area(contours)
    bounding_boxes = [calculate_bounding_box(contour) for contour in contours]
    hierarchy = [{"parent": -1, "children": []} for _ in contours]
    # Parent ve child ilişkilerini belirleme
    for i, inner_box in enumerate(bounding_boxes):
        for j, outer_box in enumerate(bounding_boxes):
            if i != j and is_inside(inner_box, outer_box):
                # Eğer `i`, `j`'nin parent'ıysa
                if hierarchy[i]["parent"] == -1:  # Zaten bir parent atanmadıysa
                    hierarchy[i]["parent"] = j
                    hierarchy[j]["children"].append(i)

    return hierarchy

# Örnek konturlar
contours = [
    [(1, 1), (1, 4), (4, 4), (4, 1)],   # Kontur 0: Dış kontur
    [(2.2, 2.2), (2.2, 2.5), (2.5, 2.5), (2.5, 2.2)],  # Kontur 1: Kontur 3'in içinde
    [(5, 5), (5, 7), (7, 7), (7, 5)],   # Kontur 2: Bağımsız
    [(2, 2), (2, 3), (3, 3), (3, 2)],    # Kontur 3: Kontur 0'ın içinde
    [(3.2, 1.5), (3.2, 1.8), (3.5, 1.8), (3.5, 1.5)]   # Kontur 4: Kontur 0'ın içinde, Kontur 3'ten bağımsız
]

hierarchy = calculate_hierarchy(contours)

# Hiyerarşi yazdırma
for i, relation in enumerate(hierarchy):
    print(f"Kontur {i}:")
    print(f"  Ebeveyn: {relation['parent']}")
    print(f"  Çocuklar: {relation['children']}")


In [ ]:
# Görüntüyü 2'ye bölme. Bunu yapma amacımız yukarıdaki görüntüde gözleri, alttaki görüntüde burun ve ağzı bulmak.
height, width = edge_dedection_grayscale_frame.shape
top_half = edge_dedection_grayscale_frame[:height // 2, :]
top_half1 = edge_dedection_grayscale_frame[:height // 2, :]
bottom_half = edge_dedection_grayscale_frame[height // 2: , :]

plt.imshow(top_half, cmap="gray")
plt.title("top_half")
plt.show()

plt.imshow(bottom_half, cmap="gray")
plt.title("bottom_half")
plt.show()

In [ ]:
# Gözlerin yerini bul
eye_regions = find_eyes(top_half)
for (x, y, w, h) in eye_regions:
#    print("**********************"*10)
#    print(x, y, w, h)
#    print("**********************"*10)
    cv2.rectangle(top_half, (x, y), (x + w, y + h), (255, 0, 0), 2)
plt.imshow(top_half, cmap='gray')
plt.title("Gözlerin Yerleri")
plt.show()


In [ ]:
def find_eyes(top_half):
    # Yoğun kontrastlı bölgeleri belirlemek için bir eşik belirleyin
    threshold = 50
    binary_top = np.where(top_half > threshold, 255, 0).astype(np.uint8)

    # Gözlerin olabileceği yuvarlak bölgeleri bul
    contours, _ = cv2.findContours(binary_top, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    eyes = []
    for contour in contours:
        # Göz boyutuna uygun konturlar seç
        x, y, w, h = cv2.boundingRect(contour)
        if 10 < w < 50 and 10 < h < 50:  # Göz boyutları için mantıklı bir aralık
            eyes.append((x, y, w, h))
    return eyes

# Gözlerin yerini bul
eye_regions = find_eyes(top_half)
for (x, y, w, h) in eye_regions:
    cv2.rectangle(top_half, (x, y), (x + w, y + h), (255, 0, 0), 2)
plt.imshow(top_half, cmap='gray')
plt.title("Gözlerin Yerleri")
plt.show()


In [ ]:
def find_eyes1(top_half):
    threshold = 50
    # Pixel değeri threshold'dan büyükse pixel'i 255, küçükse 0 yapar.
    binary_top = np.where(top_half > threshold, 255, 0).astype(np.uint8)

    # Gözlerin olabileceği yuvarlak bölgeleri bul
    contours, _ = cv2.findContours(binary_top, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#    contours = findContours(binary_top)
    eyes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if 10 < w < 50 and 10 < h < 50:  # Göz boyutları için mantıklı bir aralık
            eyes.append((x, y, w, h))
    return eyes

In [ ]:
# Gözlerin yerini bul
eye_regions = find_eyes1(top_half1)
for (x, y, w, h) in eye_regions:
    cv2.rectangle(top_half, (x, y), (x + w, y + h), (255, 0, 0), 2)
plt.imshow(top_half, cmap='gray')
plt.title("Gözlerin Yerleri")
plt.show()


# Şablon Eşleştirme

In [ ]:
import cv2
template_image = cv2.imread("./face.jpg")

frame_red = template_image[:, :, 2]
frame_green = template_image[:, :, 1]
frame_blue = template_image[:, :, 0]
frame_rgb = np.dstack((frame_red, frame_green, frame_blue))

plt.imshow(frame_rgb)
plt.title("Frame")
plt.show()

In [ ]:
template_image_grayscale = rgb_to_grayscale(frame_rgb)

plt.imshow(template_image_grayscale, cmap='gray')
plt.title("Frame_grayscale")
plt.show()

In [ ]:
filtered_template_image = gaussian_filter(template_image_grayscale, kernel_size = 7, sigma = 0.7)

plt.imshow(filtered_template_image, cmap='gray')
plt.title("filtered_grayscale_frame")
plt.show()

In [ ]:
edge_dedection_template_image = sobel_filter(filtered_template_image)

plt.imshow(edge_dedection_template_image, cmap='gray')
plt.title("edge_dedection_grayscale_frame")
plt.show()

In [ ]:
left_eye = edge_dedection_template_image[50:100, 180:220]
right_eye = edge_dedection_template_image[50:100, 240:280]
nose = edge_dedection_template_image[80:120, 210:250]
mouth = edge_dedection_template_image[125:150, 200:260]
left_eye = np.where(left_eye > 50, left_eye, 0)
plt.imshow(left_eye, cmap='gray')
plt.title("left_eye")
plt.show()

plt.imshow(right_eye, cmap='gray')
plt.title("right_eye")
plt.show()

plt.imshow(nose, cmap='gray')
plt.title("nose")
plt.show()

plt.imshow(mouth, cmap='gray')
plt.title("mouth")
plt.show()

In [ ]:
def template_matching(image, template):
    i_height, i_width = image.shape
    t_height, t_width = template.shape
    result = np.zeros((i_height - t_height + 1, i_width - t_width + 1))
    
    for y in range(i_height - t_height + 1):
        for x in range(i_width - t_width + 1):
            region = image[y:y+t_height, x:x+t_width]
            print("y=", y, "x=", x)
            plt.imshow(region, cmap = "gray")
            plt.show()
            result[y, x] = np.sum(np.abs(region - template))
    
    return result


In [ ]:
deneme = edge_dedection_grayscale_frame[220:250, 230:275]
plt.imshow(deneme, cmap="gray")
plt.show()

deneme2 = left_eye[15:33, :]
plt.imshow(deneme2, cmap="gray")
plt.show()

i_height, i_width = deneme.shape
t_height, t_width = deneme2.shape
resized_image = cv2.resize(deneme, (t_width, t_height))
plt.imshow(resized_image, cmap="gray")
plt.show()

def normalized_cross_correlation(image1, image2):
    # Normalize etmek için piksel değerlerini ortalama ve standart sapmaya göre merkezle
    image1_mean = np.mean(image1)
    image2_mean = np.mean(image2)
    image1_std = np.std(image1)
    image2_std = np.std(image2)

    # Normalize edilmiş görüntüleri oluştur
    image1_normalized = (image1 - image1_mean) / image1_std
    image2_normalized = (image2 - image2_mean) / image2_std

    # NCC değerini hesapla
    ncc_value = np.sum(image1_normalized * image2_normalized)
    ncc_value /= image1.shape[0] * image1.shape[1]  # Ortalama değer

    return ncc_value

# NCC değerini hesapla
ncc = normalized_cross_correlation(resized_image, deneme2)

# NCC değerini yüzdelik forma çevir
similarity = ncc * 100  # NCC -1 ile 1 arasında olduğundan, % olarak ifade etmek için 100 ile çarpıyoruz

print(f"Görsellerin NCC ile benzeme oranı: {similarity:.2f}%")

In [ ]:
result_left_eye = template_matching(edge_dedection_grayscale_frame, left_eye)
result_right_eye = template_matching(edge_dedection_grayscale_frame, right_eye)
result_nose = template_matching(edge_dedection_grayscale_frame, nose)
result_mouth = template_matching(edge_dedection_grayscale_frame, mouth)


In [ ]:
# Sol göz için en düşük SAD değerini bul
min_val, _, min_loc, _ = cv2.minMaxLoc(result_left_eye)
left_eye_location = min_loc  # (x, y) formatında en iyi eşleşme koordinatı

right_eye_location = cv2.minMaxLoc(result_right_eye)[2]
nose_location = cv2.minMaxLoc(result_nose)[2]
mouth_location = cv2.minMaxLoc(result_mouth)[2]


In [ ]:
# Sol göz için bir dikdörtgen çiz
x, y = left_eye_location
cv2.rectangle(edge_dedection_grayscale_frame, (x, y), (x+left_eye.shape[1], y+left_eye.shape[0]), (255, 0, 0), 2)


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(edge_dedection_grayscale_frame, cmap='gray')
plt.show()


In [ ]:
def connected_components(binary_image):
    height, width = binary_image.shape

    labels = np.zeros((height, width), dtype=np.int32)
    current_label = 1

    for y in range(height):
        for x in range(width):
            if binary_image[y][x] == 255 and labels[y][x] == 0:
                stack = [(y, x)]
                labels[y][x] = current_label

                while stack:
                    cy, cx = stack.pop()

                    for ny in range(cy - 1, cy + 2):
                        for nx in range(cx - 1, cx + 2):
                            if (0 <= ny < height) and (0 <= nx < width) and (binary_image[ny][nx] == 255) and (labels[ny][nx] == 0):
                                labels[ny][nx] = current_label
                                stack.append((ny, nx))

                current_label += 1

    return labels, current_label - 1

In [ ]:
binary_image = (edge_dedection_grayscale_frame > 0).astype(np.uint8) * 255

plt.imshow(binary_image)
plt.title("binary_image")
plt.show()

labels, num_components = connected_components(binary_image)

fig, ax = plt.subplots()
ax.imshow(binary_image, cmap='gray')
ax.set_title("Yüz Olabilecek Bölgeler")
for label in range(1, num_components + 1):
    # Bu bileşene ait piksellerin koordinatlarını bulun
    component_pixels = np.where(labels == label)
    
    # Dikdörtgenin sol üst ve sağ alt köşelerini belirleyin
    y_min, y_max = np.min(component_pixels[0]), np.max(component_pixels[0])
    x_min, x_max = np.min(component_pixels[1]), np.max(component_pixels[1])
    
    min_size = 100
    max_size = 500
    if min_size <= (x_max - x_min) <= max_size and min_size <= (y_max - y_min) <= max_size:
        # En-boy oranı filtresi
#        aspect_ratio = (x_max - x_min) / (y_max - y_min)
#        if 0.8 <= aspect_ratio <= 1.2:
            # Kenar yoğunluğu filtresi
            component_area = (y_max - y_min) * (x_max - x_min)
            edge_density = np.sum(labels[y_min:y_max+1, x_min:x_max+1] == label) / component_area
            if edge_density > 0.3:  # Kenar yoğunluğu yeterliyse yüz olarak kabul et
                # Yüz olma ihtimali yüksek bölgeye dikdörtgen çiz
                rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                     edgecolor='red', facecolor='none', linewidth=2)
                ax.add_patch(rect)
                print(f"Bölge {label} dikdörtgen olarak işaretlendi.")
                print(x_max, x_min)

In [ ]:
def sliding_window(image, window_size, step_size):
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

def multi_scale_face_detection(edge_image, initial_window_size=(150, 150), scale_factor=0.9, min_window_size=(50, 50), edge_threshold=1000):
    best_region = None
    best_edge_count = 0
    current_image = edge_image.copy()
    current_window_size = initial_window_size

    while current_image.shape[0] >= min_window_size[1] and current_image.shape[1] >= min_window_size[0]:
        # Kaydırmalı pencereyi mevcut ölçekte uygula
        for (x, y, window) in sliding_window(current_image, current_window_size, step_size=10):
            edge_count = np.sum(window)  # Kenar yoğunluğunu hesapla
            
            if edge_count > edge_threshold and edge_count > best_edge_count:
                best_edge_count = edge_count
                best_region = (x, y, current_window_size[0], current_window_size[1])

        # Görüntüyü yeniden boyutlandır (ölçeği küçült)
        current_image = current_image[::int(1/scale_factor), ::int(1/scale_factor)]
        current_window_size = (int(current_window_size[0] * scale_factor), int(current_window_size[1] * scale_factor))

    return best_region



def find_face_region(edge_image, window_size=(250, 250), step_size=10, edge_threshold=1000):
    best_region = None
    best_edge_count = 0

    for (x, y, window) in sliding_window(edge_image, window_size, step_size):
        edge_count = np.sum(window)  # Kenar yoğunluğunu hesapla

        if edge_count > edge_threshold and edge_count > best_edge_count:
            best_edge_count = edge_count
            best_region = (x, y, window_size[0], window_size[1])

    return best_region


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_face_box(image, face_regions):
    for face_region in face_regions:
        fig, ax = plt.subplots()
        ax.imshow(image, cmap='gray')
        if face_region:
            x, y, w, h = face_region
            rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
        plt.show()


a = multi_scale_face_detection(edge_dedection_grayscale_frame)
draw_face_box(edge_dedection_grayscale_frame, a)

In [ ]:
import cv2 as cv
eye_image = cv.imread('face.jpg', cv.IMREAD_COLOR)
#image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

frame_red = eye_image[:, :, 2]
frame_green = eye_image[:, :, 1]
frame_blue = eye_image[:, :, 0]
eye_rgb = np.dstack((frame_red, frame_green, frame_blue))

plt.imshow(eye_rgb)
plt.title("Frame")
plt.show()

eye_grayscale = rgb_to_grayscale(eye_rgb)

plt.imshow(eye_grayscale)
plt.title("Frame_grayscale")
plt.show()

filtered_eye = gaussian_filter(eye_grayscale, kernel_size = 3, sigma = 5)

plt.imshow(filtered_eye)
plt.title("filtered_grayscale_frame")
plt.show()

edge_dedection_eye = sobel_filter(filtered_eye)

plt.imshow(edge_dedection_eye)
plt.title("edge_dedection_grayscale_frame")
plt.show()

edge_dedection_eye_scale = edge_dedection_eye[140:180, 210:280]

plt.imshow(edge_dedection_eye_scale)
plt.title("edge_dedection_eye_scale")
plt.show()
#deneme = list([0, 0, original_image_rgb])


In [ ]:
import numpy as np

def template_matching(image, template, step_size=10):
    h, w = template.shape  # Şablon boyutları
    img_h, img_w = image.shape  # Ana görüntü boyutları
    
    best_match = None
    best_score = float('inf')  # En düşük farkı arıyoruz
    
    # Kaydırmalı pencere işlemi
    for y in range(0, img_h - h, step_size):
        for x in range(0, img_w - w, step_size):
            # Şablonun ana görüntüdeki bu bölgeyle farkını hesapla
            window = image[y:y+h, x:x+w]
            score = np.sum((window - template)**2)  # Öklidyen mesafe

            if score < best_score:
                best_score = score
                best_match = (x, y, w, h)
    
    return best_match


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_face_box(image, face_region):
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray')
    if face_region:
        x, y, w, h = face_region
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def template_matching_with_filter(image, template, step_size=10, scale_factors=[1.0, 0.9, 1.1], threshold=5e5, distance_threshold=30):
    best_matches = []
    img_h, img_w = image.shape
    
    for scale in scale_factors:
        scaled_template = cv2.resize(template, (int(template.shape[1] * scale), int(template.shape[0] * scale)))
        h, w = scaled_template.shape
        
        for y in range(0, img_h - h, step_size):
            for x in range(0, img_w - w, step_size):
                window = image[y:y+h, x:x+w]
                score = np.sum((window - scaled_template)**2)
                
                if score < threshold:
                    best_matches.append((x, y, w, h, score))
    
    # Skorlara göre sırala ve belirli bir mesafede olanları birleştir
    best_matches = sorted(best_matches, key=lambda x: x[4])
    filtered_matches = merge_close_matches(best_matches, distance_threshold)
    
    return filtered_matches[:7]

def merge_close_matches(matches, distance_threshold=30):
    merged_matches = []
    for (x, y, w, h, score) in matches:
        merged = False
        for i, (mx, my, mw, mh, mscore) in enumerate(merged_matches):
            if abs(mx - x) < distance_threshold and abs(my - y) < distance_threshold:
                merged = True
                break
        if not merged:
            merged_matches.append((x, y, w, h, score))
    return merged_matches

# Görselleştirme fonksiyonu
def draw_face_boxes(image, face_regions):
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray')
    for (x, y, w, h, score) in face_regions:
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()

import cv2

def apply_threshold(image, threshold_value=100):
    _, thresholded_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return thresholded_image



# Kenar görüntüsüne threshold uygulayın
edge_image = apply_threshold(edge_dedection_grayscale_frame)

In [ ]:
a = template_matching_with_filter(edge_image, edge_dedection_eye_scale)
draw_face_boxes(edge_dedection_grayscale_frame, a)

In [ ]:
edge_dedection_grayscale_frame_resize = edge_dedection_grayscale_frame[40:420, 180:430]

plt.imshow(edge_dedection_grayscale_frame_resize)
plt.title("edge_dedection_grayscale_frame")
plt.show()

edge_dedection_grayscale_frame_resize = cv2.resize(edge_dedection_grayscale_frame_resize, (185, 265))

plt.imshow(edge_dedection_grayscale_frame_resize)
plt.title("edge_dedection_grayscale_frame")
plt.show()

In [ ]:
import cv2 as cv
original_image = cv.imread('./test/youtube-43_jpg.rf.5806bace236e1162900bf79aeaf44f36.jpg', cv.IMREAD_COLOR)
#image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

frame_red = original_image[:, :, 2]
frame_green = original_image[:, :, 1]
frame_blue = original_image[:, :, 0]
original_image_rgb = np.dstack((frame_red, frame_green, frame_blue))

plt.imshow(original_image_rgb)
plt.title("Frame")
plt.show()

#image = rgb_to_grayscale(image)
original_image_rgb = original_image_rgb[10:275, 210:395]
#original_image_rgb = cv2.resize(original_image_rgb, (64, 128))


original_image_grayscale = rgb_to_grayscale(original_image_rgb)

plt.imshow(original_image_grayscale)
plt.title("Frame_grayscale")
plt.show()

filtered_original_image = gaussian_filter(original_image_grayscale, kernel_size = 3, sigma = 5)

plt.imshow(filtered_original_image)
plt.title("filtered_grayscale_frame")
plt.show()

edge_dedection_original_image = sobel_filter(filtered_original_image)

plt.imshow(edge_dedection_original_image)
plt.title("edge_dedection_grayscale_frame")
plt.show()

#deneme = list([0, 0, original_image_rgb])


In [ ]:
I1 = edge_dedection_grayscale_frame_resize.flatten()
I2 = edge_dedection_original_image.flatten()

I1_mean = np.mean(I1)
I2_mean = np.mean(I2)

numerator = np.sum((I1 - I1_mean) * (I2 - I2_mean))
denominator = np.sqrt(np.sum((I1 - I1_mean) ** 2) * np.sum((I2 - I2_mean) ** 2))

correlation = numerator / denominator

print(correlation)